In [1]:
import numpy as np

from core.TrajectoryDF import NumPandasTraj as NumTrajDF
from features.spatial_features import SpatialFeatures as spatial
from utilities.conversions import Conversions as con
import pandas as pd
import time
np.seterr(invalid='ignore')
start = time.time()

In [2]:
%%time
# Reading the geolife dataset and converting to NumPandasTraj.
geolife = pd.read_csv('./data/geolife_sample.csv')
geolife = NumTrajDF(geolife,'lat','lon','datetime','id')

# Reading the gulls dataset and converting to NumPandasTraj.
gulls = pd.read_csv('./data/gulls.csv')
gulls = NumTrajDF(gulls,
                 latitude='location-lat',
                 longitude='location-long',
                 datetime='timestamp',
                 traj_id='tag-local-identifier',
                 rest_of_columns=[])

# Reading the atlantic dataset, cleaning it up and then converting
# it to NumPandasTraj.
atlantic = pd.read_csv('./data/atlantic.csv')
atlantic = con.convert_directions_to_degree_lat_lon(atlantic, 'Latitude',"Longitude")
def convert_to_datetime(row):
        this_date = '{}-{}-{}'.format(str(row['Date'])[0:4], str(row['Date'])[4:6], str(row['Date'])[6:])
        this_time = '{:02d}:{:02d}:00'.format(int(row['Time']/100), int(str(row['Time'])[-2:]))
        return '{} {}'.format(this_date, this_time)
atlantic['DateTime'] = atlantic.apply(convert_to_datetime, axis=1)
atlantic = NumTrajDF(atlantic,
                         latitude='Latitude',
                         longitude='Longitude',
                         datetime='DateTime',
                         traj_id='ID',
                         rest_of_columns=[])
atlantic.head()

CPU times: user 9.85 s, sys: 121 ms, total: 9.97 s
Wall time: 9.97 s


Name      Date  Time Event  \
traj_id  DateTime                                                         
AL011851 1851-06-25 00:00:00              UNNAMED  18510625     0         
         1851-06-25 06:00:00              UNNAMED  18510625   600         
         1851-06-25 12:00:00              UNNAMED  18510625  1200         
         1851-06-25 18:00:00              UNNAMED  18510625  1800         
         1851-06-25 21:00:00              UNNAMED  18510625  2100     L   

                             Status   lat   lon  Maximum Wind  \
traj_id  DateTime                                               
AL011851 1851-06-25 00:00:00     HU  28.0 -94.8            80   
         1851-06-25 06:00:00     HU  28.0 -95.4            80   
         1851-06-25 12:00:00     HU  28.0 -96.0            80   
         1851-06-25 18:00:00     HU  28.1 -96.5            80   
         1851-06-25 21:00:00     HU  28.2 -96.8            80   

                              Minimum Pressure  Low Wind NE  ...  Low Wind SW  \
traj_id  DateTime                                            ...                
AL011851 1851-06-25 00:00:00              -999         -999  ...         -999   
         1851-06-25 06:00:00              -999         -999  ...         -999   
         1851-06-25 12:00:00              -999         -999  ...         -999   
         1851-06-25 18:00:00              -999         -999  ...         -999   
         1851-06-25 21:00:00              -999         -999  ...         -999   

                              Low Wind NW  Moderate Wind NE  Moderate Wind SE  \
traj_id  DateTime                                                               
AL011851 1851-06-25 00:00:00         -999              -999              -999   
         1851-06-25 06:00:00         -999              -999              -999   
         1851-06-25 12:00:00         -999              -999              -999   
         1851-06-25 18:00:00         -999              -999              -999   
         1851-06-25 21:00:00         -999              -999              -999   

                              Moderate Wind SW  Moderate Wind NW  \
traj_id  DateTime                                                  
AL011851 1851-06-25 00:00:00              -999              -999   
         1851-06-25 06:00:00              -999              -999   
         1851-06-25 12:00:00              -999              -999   
         1851-06-25 18:00:00              -999              -999   
         1851-06-25 21:00:00              -999              -999   

                              High Wind NE  High Wind SE  High Wind SW  \
traj_id  DateTime                                                        
AL011851 1851-06-25 00:00:00          -999          -999          -999   
         1851-06-25 06:00:00          -999          -999          -999   
         1851-06-25 12:00:00          -999          -999          -999   
         1851-06-25 18:00:00          -999          -999          -999   
         1851-06-25 21:00:00          -999          -999          -999   

                              High Wind NW  
traj_id  DateTime                           
AL011851 1851-06-25 00:00:00          -999  
         1851-06-25 06:00:00          -999  
         1851-06-25 12:00:00          -999  
         1851-06-25 18:00:00          -999  
         1851-06-25 21:00:00          -999  

[5 rows x 21 columns]

In [3]:
%%time

# Now, lets get the bounding boxes of all the dataframes.
print(spatial.get_bounding_box(geolife))
print(spatial.get_bounding_box(gulls))
print(spatial.get_bounding_box(atlantic))

(22.147577, 113.54884299999999, 41.132062, 121.156224)
(-62.71617, 4.02517, 65.2325, 103.19317)
(7.2, -109.5, 81.0, 63.0)
CPU times: user 9.03 ms, sys: 64 µs, total: 9.09 ms
Wall time: 7.68 ms


In [4]:
%%time

# Now, lets get the start times of all the dataframes without
# giving a traj_id.
print(spatial.get_start_location(geolife))
print(spatial.get_start_location(gulls))
print(spatial.get_start_location(atlantic))

               lat         lon
traj_id                       
1        39.984224  116.319402
5        40.004155  116.321337
              lat       lon
traj_id                    
91785    64.98267  35.74300
91797    64.96767  35.73650
91769    54.18433   7.91283
91843    55.42533  49.34817
91913    64.97850  35.72667
...           ...       ...
91814    55.43233  49.34283
91763    62.78917  28.76983
91823    54.18600   7.91867
91752    62.40617  29.46983
91916    54.18533   7.91500

[126 rows x 2 columns]
           lat   lon
traj_id             
AL021864  24.0 -89.0
AL011853  32.5 -69.0
AL021865  26.0 -97.5
AL071870  12.0 -35.0
AL041856  23.0 -82.5
...        ...   ...
AL041957  13.0 -21.3
AL142012  15.5 -38.0
AL201969  16.0 -47.0
AL141971  25.5 -71.5
AL031899  11.7 -31.0

[1814 rows x 2 columns]
CPU times: user 628 ms, sys: 374 ms, total: 1 s
Wall time: 5.12 s


In [5]:
%%time

# Now lets check the start times of all dataframes with a traj_id.
print(spatial.get_start_location(geolife, '1'))
print(spatial.get_start_location(gulls, "91732"))
print(spatial.get_start_location(atlantic, 'AL011851'))

(39.984224, 116.319402)
(61.24783000000001, 24.586170000000003)
(28.0, -94.8)
CPU times: user 125 ms, sys: 15.8 ms, total: 141 ms
Wall time: 139 ms


In [6]:
%%time

# Now, lets get the start times of all the dataframes without
# giving a traj_id.
print(spatial.get_end_location(geolife))
print(spatial.get_end_location(gulls))
print(spatial.get_end_location(atlantic))

               lat         lon
traj_id                       
1        39.977879  116.326628
5        39.999978  116.327460
              lat       lon
traj_id                    
91785    64.98267  35.74300
91797    64.96767  35.73633
91769    54.18417   7.91350
91843    55.43533  49.36383
91913    64.96700  35.74083
...           ...       ...
91814    -1.79117  32.80583
91763     6.97833  29.51200
91823    61.31100  24.57767
91752     8.06067  38.85033
91916    56.56600  21.19633

[126 rows x 2 columns]
           lat   lon
traj_id             
AL021864  24.0 -89.0
AL011853  32.5 -69.0
AL021865  26.0 -97.5
AL071870  12.0 -35.0
AL041856  23.0 -82.5
...        ...   ...
AL041957  50.0  -6.0
AL142012  37.6 -29.1
AL201969  26.1 -61.6
AL141971  35.7 -69.9
AL031899  49.0 -15.5

[1814 rows x 2 columns]
CPU times: user 578 ms, sys: 453 ms, total: 1.03 s
Wall time: 5.35 s


In [7]:
%%time

# Now lets check the start times of all dataframes with a traj_id.
print(spatial.get_end_location(geolife, '1'))
print(spatial.get_end_location(gulls, "91732"))
print(spatial.get_end_location(atlantic, 'AL011851'))

(39.977878999999994, 116.326628)
(31.09783, 29.812170000000002)
(31.0, -100.2)
CPU times: user 92.4 ms, sys: 0 ns, total: 92.4 ms
Wall time: 90.9 ms


In [8]:
%%time

# Now, lets calculate the distance between 2 consecutive points for
# each dataset one by one.
# First, we will check the distances between 2 consecutive points
# in the geolife dataset.
geolife = spatial.create_distance_between_consecutive_column(geolife)
geolife.head(10)

CPU times: user 279 ms, sys: 11.8 ms, total: 291 ms
Wall time: 290 ms


lat         lon  Distance_prev_to_curr
traj_id DateTime                                                         
1       2008-10-23 16:53:05  39.984094  116.319236                    NaN
        2008-10-23 16:53:06  39.984198  116.319322              13.690153
        2008-10-23 05:53:11  39.984224  116.319402               7.403788
        2008-10-23 05:53:16  39.984211  116.319389               1.821083
        2008-10-23 05:53:21  39.984217  116.319422               2.889671
        2008-10-23 05:53:23  39.984710  116.319865              66.555997
        2008-10-23 05:53:28  39.984674  116.319810               6.162987
        2008-10-23 05:53:33  39.984623  116.319773               6.488225
        2008-10-23 05:53:38  39.984606  116.319732               3.971848
        2008-10-23 05:53:43  39.984555  116.319728               5.681172

In [9]:
%%time

# # Here, we check the distance between 2 consecutive points
# # in the seagulls dataset.
gulls = spatial.create_distance_between_consecutive_column(gulls)
gulls.head()

CPU times: user 208 ms, sys: 64 ms, total: 272 ms
Wall time: 589 ms


event-id  visible       lon       lat  \
traj_id DateTime                                                       
91732   2009-05-27 14:00:00  1082620685     True  24.58617  61.24783   
        2009-05-27 20:00:00  1082620686     True  24.58217  61.23267   
        2009-05-28 05:00:00  1082620687     True  24.53133  61.18833   
        2009-05-28 08:00:00  1082620688     True  24.58200  61.23283   
        2009-05-28 14:00:00  1082620689     True  24.58250  61.23267   

                            sensor-type individual-taxon-canonical-name  \
traj_id DateTime                                                          
91732   2009-05-27 14:00:00         gps                    Larus fuscus   
        2009-05-27 20:00:00         gps                    Larus fuscus   
        2009-05-28 05:00:00         gps                    Larus fuscus   
        2009-05-28 08:00:00         gps                    Larus fuscus   
        2009-05-28 14:00:00         gps                    Larus fuscus   

                            individual-local-identifier  \
traj_id DateTime                                          
91732   2009-05-27 14:00:00                      91732A   
        2009-05-27 20:00:00                      91732A   
        2009-05-28 05:00:00                      91732A   
        2009-05-28 08:00:00                      91732A   
        2009-05-28 14:00:00                      91732A   

                                                                    study-name  \
traj_id DateTime                                                                 
91732   2009-05-27 14:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-27 20:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 05:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 08:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 14:00:00  Navigation experiments in lesser black-backed ...   

                             Distance_prev_to_curr  
traj_id DateTime                                    
91732   2009-05-27 14:00:00                    NaN  
        2009-05-27 20:00:00            1699.244398  
        2009-05-28 05:00:00            5632.120064  
        2009-05-28 08:00:00            5643.314949  
        2009-05-28 14:00:00              32.131494

In [10]:
%%time

# Here, we check the distance between 2 consecutive points
# in the atlantic dataset.
atlantic = spatial.create_distance_between_consecutive_column(atlantic)
atlantic.head()

CPU times: user 322 ms, sys: 195 ms, total: 517 ms
Wall time: 1.14 s


Name      Date  Time Event  \
traj_id  DateTime                                                         
AL021873 1873-08-13 00:00:00              UNNAMED  18730813     0         
         1873-08-13 06:00:00              UNNAMED  18730813   600         
         1873-08-13 12:00:00              UNNAMED  18730813  1200         
         1873-08-13 18:00:00              UNNAMED  18730813  1800         
         1873-08-14 00:00:00              UNNAMED  18730814     0         

                             Status   lat   lon  Maximum Wind  \
traj_id  DateTime                                               
AL021873 1873-08-13 00:00:00     TS  13.9 -25.0            40   
         1873-08-13 06:00:00     TS  14.0 -26.0            40   
         1873-08-13 12:00:00     TS  14.0 -27.0            40   
         1873-08-13 18:00:00     TS  14.1 -28.1            40   
         1873-08-14 00:00:00     TS  14.1 -29.3            50   

                              Minimum Pressure  Low Wind NE  ...  Low Wind NW  \
traj_id  DateTime                                            ...                
AL021873 1873-08-13 00:00:00              -999         -999  ...         -999   
         1873-08-13 06:00:00              -999         -999  ...         -999   
         1873-08-13 12:00:00              -999         -999  ...         -999   
         1873-08-13 18:00:00              -999         -999  ...         -999   
         1873-08-14 00:00:00              -999         -999  ...         -999   

                              Moderate Wind NE  Moderate Wind SE  \
traj_id  DateTime                                                  
AL021873 1873-08-13 00:00:00              -999              -999   
         1873-08-13 06:00:00              -999              -999   
         1873-08-13 12:00:00              -999              -999   
         1873-08-13 18:00:00              -999              -999   
         1873-08-14 00:00:00              -999              -999   

                              Moderate Wind SW  Moderate Wind NW  \
traj_id  DateTime                                                  
AL021873 1873-08-13 00:00:00              -999              -999   
         1873-08-13 06:00:00              -999              -999   
         1873-08-13 12:00:00              -999              -999   
         1873-08-13 18:00:00              -999              -999   
         1873-08-14 00:00:00              -999              -999   

                              High Wind NE  High Wind SE  High Wind SW  \
traj_id  DateTime                                                        
AL021873 1873-08-13 00:00:00          -999          -999          -999   
         1873-08-13 06:00:00          -999          -999          -999   
         1873-08-13 12:00:00          -999          -999          -999   
         1873-08-13 18:00:00          -999          -999          -999   
         1873-08-14 00:00:00          -999          -999          -999   

                              High Wind NW  Distance_prev_to_curr  
traj_id  DateTime                                                  
AL021873 1873-08-13 00:00:00          -999                    NaN  
         1873-08-13 06:00:00          -999          108486.658872  
         1873-08-13 12:00:00          -999          107891.881958  
         1873-08-13 18:00:00          -999          119175.045002  
         1873-08-14 00:00:00          -999          129413.676731  

[5 rows x 22 columns]

In [11]:
%%time

# Now, lets calculate the distance from start to current point for
# each dataset one by one.
# Here, we check the distance between start to all points
# in the geolife dataset.

geolife = spatial.create_distance_from_start_column(geolife)
geolife.head()

CPU times: user 232 ms, sys: 7.9 ms, total: 240 ms
Wall time: 238 ms


lat         lon  Distance_prev_to_curr  \
traj_id DateTime                                                            
1       2008-10-23 16:53:05  39.984094  116.319236                    NaN   
        2008-10-23 16:53:06  39.984198  116.319322              13.690153   
        2008-10-23 05:53:11  39.984224  116.319402               7.403788   
        2008-10-23 05:53:16  39.984211  116.319389               1.821083   
        2008-10-23 05:53:21  39.984217  116.319422               2.889671   

                             Distance_start_to_curr  
traj_id DateTime                                     
1       2008-10-23 16:53:05                     NaN  
        2008-10-23 16:53:06               13.690153  
        2008-10-23 05:53:11               20.223428  
        2008-10-23 05:53:16               18.416895  
        2008-10-23 05:53:21               20.933073

In [12]:
%%time

# Here, we check the distance between start to all points
# in the seagulls dataset.
gulls = spatial.create_distance_from_start_column(gulls)
gulls.head()

CPU times: user 154 ms, sys: 63.4 ms, total: 218 ms
Wall time: 458 ms


event-id  visible       lon       lat  \
traj_id DateTime                                                       
91732   2009-05-27 14:00:00  1082620685     True  24.58617  61.24783   
        2009-05-27 20:00:00  1082620686     True  24.58217  61.23267   
        2009-05-28 05:00:00  1082620687     True  24.53133  61.18833   
        2009-05-28 08:00:00  1082620688     True  24.58200  61.23283   
        2009-05-28 14:00:00  1082620689     True  24.58250  61.23267   

                            sensor-type individual-taxon-canonical-name  \
traj_id DateTime                                                          
91732   2009-05-27 14:00:00         gps                    Larus fuscus   
        2009-05-27 20:00:00         gps                    Larus fuscus   
        2009-05-28 05:00:00         gps                    Larus fuscus   
        2009-05-28 08:00:00         gps                    Larus fuscus   
        2009-05-28 14:00:00         gps                    Larus fuscus   

                            individual-local-identifier  \
traj_id DateTime                                          
91732   2009-05-27 14:00:00                      91732A   
        2009-05-27 20:00:00                      91732A   
        2009-05-28 05:00:00                      91732A   
        2009-05-28 08:00:00                      91732A   
        2009-05-28 14:00:00                      91732A   

                                                                    study-name  \
traj_id DateTime                                                                 
91732   2009-05-27 14:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-27 20:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 05:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 08:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 14:00:00  Navigation experiments in lesser black-backed ...   

                             Distance_prev_to_curr  Distance_start_to_curr  
traj_id DateTime                                                            
91732   2009-05-27 14:00:00                    NaN                     NaN  
        2009-05-27 20:00:00            1699.244398             1699.244398  
        2009-05-28 05:00:00            5632.120064             7238.295521  
        2009-05-28 08:00:00            5643.314949             1682.777895  
        2009-05-28 14:00:00              32.131494             1697.111327

In [13]:
%%time

# Here, we check the distance between start to all points
# in the atlantic dataset.
atlantic = spatial.create_distance_from_start_column(atlantic)
atlantic.head()

CPU times: user 343 ms, sys: 139 ms, total: 482 ms
Wall time: 1.16 s


Name      Date  Time Event  \
traj_id  DateTime                                                         
AL021873 1873-08-13 00:00:00              UNNAMED  18730813     0         
         1873-08-13 06:00:00              UNNAMED  18730813   600         
         1873-08-13 12:00:00              UNNAMED  18730813  1200         
         1873-08-13 18:00:00              UNNAMED  18730813  1800         
         1873-08-14 00:00:00              UNNAMED  18730814     0         

                             Status   lat   lon  Maximum Wind  \
traj_id  DateTime                                               
AL021873 1873-08-13 00:00:00     TS  13.9 -25.0            40   
         1873-08-13 06:00:00     TS  14.0 -26.0            40   
         1873-08-13 12:00:00     TS  14.0 -27.0            40   
         1873-08-13 18:00:00     TS  14.1 -28.1            40   
         1873-08-14 00:00:00     TS  14.1 -29.3            50   

                              Minimum Pressure  Low Wind NE  ...  \
traj_id  DateTime                                            ...   
AL021873 1873-08-13 00:00:00              -999         -999  ...   
         1873-08-13 06:00:00              -999         -999  ...   
         1873-08-13 12:00:00              -999         -999  ...   
         1873-08-13 18:00:00              -999         -999  ...   
         1873-08-14 00:00:00              -999         -999  ...   

                              Moderate Wind NE  Moderate Wind SE  \
traj_id  DateTime                                                  
AL021873 1873-08-13 00:00:00              -999              -999   
         1873-08-13 06:00:00              -999              -999   
         1873-08-13 12:00:00              -999              -999   
         1873-08-13 18:00:00              -999              -999   
         1873-08-14 00:00:00              -999              -999   

                              Moderate Wind SW  Moderate Wind NW  \
traj_id  DateTime                                                  
AL021873 1873-08-13 00:00:00              -999              -999   
         1873-08-13 06:00:00              -999              -999   
         1873-08-13 12:00:00              -999              -999   
         1873-08-13 18:00:00              -999              -999   
         1873-08-14 00:00:00              -999              -999   

                              High Wind NE  High Wind SE  High Wind SW  \
traj_id  DateTime                                                        
AL021873 1873-08-13 00:00:00          -999          -999          -999   
         1873-08-13 06:00:00          -999          -999          -999   
         1873-08-13 12:00:00          -999          -999          -999   
         1873-08-13 18:00:00          -999          -999          -999   
         1873-08-14 00:00:00          -999          -999          -999   

                              High Wind NW  Distance_prev_to_curr  \
traj_id  DateTime                                                   
AL021873 1873-08-13 00:00:00          -999                    NaN   
         1873-08-13 06:00:00          -999          108486.658872   
         1873-08-13 12:00:00          -999          107891.881958   
         1873-08-13 18:00:00          -999          119175.045002   
         1873-08-14 00:00:00          -999          129413.676731   

                              Distance_start_to_curr  
traj_id  DateTime                                     
AL021873 1873-08-13 00:00:00                     NaN  
         1873-08-13 06:00:00           108486.658872  
         1873-08-13 12:00:00           216116.369259  
         1873-08-13 18:00:00           335201.031514  
         1873-08-14 00:00:00           464461.502447  

[5 rows x 23 columns]

In [14]:
%%time

# Here, we are calculating the distance travelled in traj_id 1
# on the date 2008-10-23.
spatial.get_distance_travelled_by_date_and_traj_id(geolife,
                                                   '2008-10-23',
                                                   '1')

CPU times: user 172 ms, sys: 0 ns, total: 172 ms
Wall time: 171 ms


14748.278545594158

In [15]:
%%time

# Now, lets check the distance of all the points
# in the geolife dataset from the coordinates (0, 0).
geolife = spatial.create_distance_from_given_point_column(geolife,
                                                          (0, 0))
geolife.head()

CPU times: user 377 ms, sys: 80.3 ms, total: 458 ms
Wall time: 3.75 s


lat         lon  Distance_prev_to_curr  \
traj_id DateTime                                                            
5       2008-10-24 04:12:30  40.004155  116.321337                    NaN   
        2008-10-24 04:12:35  40.003834  116.321462              37.247654   
        2008-10-24 04:12:40  40.003783  116.321431               6.255518   
        2008-10-24 04:12:45  40.003690  116.321429              10.342531   
        2008-10-24 04:12:50  40.003589  116.321427              11.231979   

                             Distance_start_to_curr  Distance_to_(0, 0)  
traj_id DateTime                                                         
5       2008-10-24 04:12:30                     NaN        1.221536e+07  
        2008-10-24 04:12:35               37.247654        1.221538e+07  
        2008-10-24 04:12:40               42.132251        1.221538e+07  
        2008-10-24 04:12:45               52.296064        1.221538e+07  
        2008-10-24 04:12:50               63.401465        1.221539e+07

In [16]:
%%time

# Now, lets check if all the points in the geolife dataset
# are within 12000 KMs of the coordinates (0, 0).
geolife = spatial.create_point_within_range_column(geolife,
                                                   (0,0),
                                                   15000000)
geolife.head()

CPU times: user 353 ms, sys: 98.8 ms, total: 452 ms
Wall time: 3.23 s


lat         lon  Distance_prev_to_curr  \
traj_id DateTime                                                            
5       2008-10-24 04:12:30  40.004155  116.321337                    NaN   
        2008-10-24 04:12:35  40.003834  116.321462              37.247654   
        2008-10-24 04:12:40  40.003783  116.321431               6.255518   
        2008-10-24 04:12:45  40.003690  116.321429              10.342531   
        2008-10-24 04:12:50  40.003589  116.321427              11.231979   

                             Distance_start_to_curr  Distance_to_(0, 0)  \
traj_id DateTime                                                          
5       2008-10-24 04:12:30                     NaN        1.221536e+07   
        2008-10-24 04:12:35               37.247654        1.221538e+07   
        2008-10-24 04:12:40               42.132251        1.221538e+07   
        2008-10-24 04:12:45               52.296064        1.221538e+07   
        2008-10-24 04:12:50               63.401465        1.221539e+07   

                             Within_15000000_m_from_(0, 0)  
traj_id DateTime                                            
5       2008-10-24 04:12:30                           True  
        2008-10-24 04:12:35                           True  
        2008-10-24 04:12:40                           True  
        2008-10-24 04:12:45                           True  
        2008-10-24 04:12:50                           True

In [17]:
%%time

# Now, lets calculate the speed of the object between 2
# consecutive trajectory points.
# First, we are doing in the geolife dataset.
geolife = spatial.create_speed_from_prev_column(geolife)
geolife.head()

CPU times: user 25.5 ms, sys: 4.22 ms, total: 29.7 ms
Wall time: 27.7 ms


lat         lon  Distance_prev_to_curr  \
traj_id DateTime                                                            
5       2008-10-24 04:12:30  40.004155  116.321337                    NaN   
        2008-10-24 04:12:35  40.003834  116.321462              37.247654   
        2008-10-24 04:12:40  40.003783  116.321431               6.255518   
        2008-10-24 04:12:45  40.003690  116.321429              10.342531   
        2008-10-24 04:12:50  40.003589  116.321427              11.231979   

                             Distance_start_to_curr  Distance_to_(0, 0)  \
traj_id DateTime                                                          
5       2008-10-24 04:12:30                     NaN        1.221536e+07   
        2008-10-24 04:12:35               37.247654        1.221538e+07   
        2008-10-24 04:12:40               42.132251        1.221538e+07   
        2008-10-24 04:12:45               52.296064        1.221538e+07   
        2008-10-24 04:12:50               63.401465        1.221539e+07   

                             Within_15000000_m_from_(0, 0)  Speed_prev_to_curr  
traj_id DateTime                                                                
5       2008-10-24 04:12:30                           True                 NaN  
        2008-10-24 04:12:35                           True            7.449531  
        2008-10-24 04:12:40                           True            1.251104  
        2008-10-24 04:12:45                           True            2.068506  
        2008-10-24 04:12:50                           True            2.246396

In [18]:
%%time

# Now lets calculate the speed between 2 consecutive
# points in the atlantic dataset.
atlantic = spatial.create_speed_from_prev_column(atlantic)
atlantic.head()

CPU times: user 12.9 ms, sys: 125 µs, total: 13 ms
Wall time: 11.8 ms


Name      Date  Time Event  \
traj_id  DateTime                                                         
AL021873 1873-08-13 00:00:00              UNNAMED  18730813     0         
         1873-08-13 06:00:00              UNNAMED  18730813   600         
         1873-08-13 12:00:00              UNNAMED  18730813  1200         
         1873-08-13 18:00:00              UNNAMED  18730813  1800         
         1873-08-14 00:00:00              UNNAMED  18730814     0         

                             Status   lat   lon  Maximum Wind  \
traj_id  DateTime                                               
AL021873 1873-08-13 00:00:00     TS  13.9 -25.0            40   
         1873-08-13 06:00:00     TS  14.0 -26.0            40   
         1873-08-13 12:00:00     TS  14.0 -27.0            40   
         1873-08-13 18:00:00     TS  14.1 -28.1            40   
         1873-08-14 00:00:00     TS  14.1 -29.3            50   

                              Minimum Pressure  Low Wind NE  ...  \
traj_id  DateTime                                            ...   
AL021873 1873-08-13 00:00:00              -999         -999  ...   
         1873-08-13 06:00:00              -999         -999  ...   
         1873-08-13 12:00:00              -999         -999  ...   
         1873-08-13 18:00:00              -999         -999  ...   
         1873-08-14 00:00:00              -999         -999  ...   

                              Moderate Wind SE  Moderate Wind SW  \
traj_id  DateTime                                                  
AL021873 1873-08-13 00:00:00              -999              -999   
         1873-08-13 06:00:00              -999              -999   
         1873-08-13 12:00:00              -999              -999   
         1873-08-13 18:00:00              -999              -999   
         1873-08-14 00:00:00              -999              -999   

                              Moderate Wind NW  High Wind NE  High Wind SE  \
traj_id  DateTime                                                            
AL021873 1873-08-13 00:00:00              -999          -999          -999   
         1873-08-13 06:00:00              -999          -999          -999   
         1873-08-13 12:00:00              -999          -999          -999   
         1873-08-13 18:00:00              -999          -999          -999   
         1873-08-14 00:00:00              -999          -999          -999   

                              High Wind SW  High Wind NW  \
traj_id  DateTime                                          
AL021873 1873-08-13 00:00:00          -999          -999   
         1873-08-13 06:00:00          -999          -999   
         1873-08-13 12:00:00          -999          -999   
         1873-08-13 18:00:00          -999          -999   
         1873-08-14 00:00:00          -999          -999   

                              Distance_prev_to_curr  Distance_start_to_curr  \
traj_id  DateTime                                                             
AL021873 1873-08-13 00:00:00                    NaN                     NaN   
         1873-08-13 06:00:00          108486.658872           108486.658872   
         1873-08-13 12:00:00          107891.881958           216116.369259   
         1873-08-13 18:00:00          119175.045002           335201.031514   
         1873-08-14 00:00:00          129413.676731           464461.502447   

                              Speed_prev_to_curr  
traj_id  DateTime                                 
AL021873 1873-08-13 00:00:00                 NaN  
         1873-08-13 06:00:00            5.022531  
         1873-08-13 12:00:00            4.994995  
         1873-08-13 18:00:00            5.517363  
         1873-08-14 00:00:00            5.991374  

[5 rows x 24 columns]

In [19]:
%%time

# Now lets calculate the speed between 2 consecutive
# points in the seagulls dataset.
gulls = spatial.create_speed_from_prev_column(gulls)
gulls.head()

CPU times: user 11.5 ms, sys: 3.34 ms, total: 14.8 ms
Wall time: 13.9 ms


event-id  visible       lon       lat  \
traj_id DateTime                                                       
91732   2009-05-27 14:00:00  1082620685     True  24.58617  61.24783   
        2009-05-27 20:00:00  1082620686     True  24.58217  61.23267   
        2009-05-28 05:00:00  1082620687     True  24.53133  61.18833   
        2009-05-28 08:00:00  1082620688     True  24.58200  61.23283   
        2009-05-28 14:00:00  1082620689     True  24.58250  61.23267   

                            sensor-type individual-taxon-canonical-name  \
traj_id DateTime                                                          
91732   2009-05-27 14:00:00         gps                    Larus fuscus   
        2009-05-27 20:00:00         gps                    Larus fuscus   
        2009-05-28 05:00:00         gps                    Larus fuscus   
        2009-05-28 08:00:00         gps                    Larus fuscus   
        2009-05-28 14:00:00         gps                    Larus fuscus   

                            individual-local-identifier  \
traj_id DateTime                                          
91732   2009-05-27 14:00:00                      91732A   
        2009-05-27 20:00:00                      91732A   
        2009-05-28 05:00:00                      91732A   
        2009-05-28 08:00:00                      91732A   
        2009-05-28 14:00:00                      91732A   

                                                                    study-name  \
traj_id DateTime                                                                 
91732   2009-05-27 14:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-27 20:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 05:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 08:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 14:00:00  Navigation experiments in lesser black-backed ...   

                             Distance_prev_to_curr  Distance_start_to_curr  \
traj_id DateTime                                                             
91732   2009-05-27 14:00:00                    NaN                     NaN   
        2009-05-27 20:00:00            1699.244398             1699.244398   
        2009-05-28 05:00:00            5632.120064             7238.295521   
        2009-05-28 08:00:00            5643.314949             1682.777895   
        2009-05-28 14:00:00              32.131494             1697.111327   

                             Speed_prev_to_curr  
traj_id DateTime                                 
91732   2009-05-27 14:00:00                 NaN  
        2009-05-27 20:00:00            0.078669  
        2009-05-28 05:00:00            0.173831  
        2009-05-28 08:00:00            0.522529  
        2009-05-28 14:00:00            0.001488

In [20]:
%%time

# Now lets check the acceleration of the object
# between 2 consecutive points.
# First, lets check for the geolife dataset.
geolife = spatial.create_acceleration_from_prev_column(geolife)
geolife.head()

CPU times: user 27 ms, sys: 420 µs, total: 27.4 ms
Wall time: 26.2 ms


lat         lon  Distance_prev_to_curr  \
traj_id DateTime                                                            
5       2008-10-24 04:12:30  40.004155  116.321337                    NaN   
        2008-10-24 04:12:35  40.003834  116.321462              37.247654   
        2008-10-24 04:12:40  40.003783  116.321431               6.255518   
        2008-10-24 04:12:45  40.003690  116.321429              10.342531   
        2008-10-24 04:12:50  40.003589  116.321427              11.231979   

                             Distance_start_to_curr  Distance_to_(0, 0)  \
traj_id DateTime                                                          
5       2008-10-24 04:12:30                     NaN        1.221536e+07   
        2008-10-24 04:12:35               37.247654        1.221538e+07   
        2008-10-24 04:12:40               42.132251        1.221538e+07   
        2008-10-24 04:12:45               52.296064        1.221538e+07   
        2008-10-24 04:12:50               63.401465        1.221539e+07   

                             Within_15000000_m_from_(0, 0)  \
traj_id DateTime                                             
5       2008-10-24 04:12:30                           True   
        2008-10-24 04:12:35                           True   
        2008-10-24 04:12:40                           True   
        2008-10-24 04:12:45                           True   
        2008-10-24 04:12:50                           True   

                             Speed_prev_to_curr  Acceleration_prev_to_curr  
traj_id DateTime                                                            
5       2008-10-24 04:12:30                 NaN                        NaN  
        2008-10-24 04:12:35            7.449531                        NaN  
        2008-10-24 04:12:40            1.251104                  -1.239685  
        2008-10-24 04:12:45            2.068506                   0.163481  
        2008-10-24 04:12:50            2.246396                   0.035578

In [21]:
%%time

# Now lets check the acceleration of the gulls dataset.
gulls = spatial.create_acceleration_from_prev_column(gulls)
gulls.head()

CPU times: user 20.6 ms, sys: 0 ns, total: 20.6 ms
Wall time: 19.2 ms


event-id  visible       lon       lat  \
traj_id DateTime                                                       
91732   2009-05-27 14:00:00  1082620685     True  24.58617  61.24783   
        2009-05-27 20:00:00  1082620686     True  24.58217  61.23267   
        2009-05-28 05:00:00  1082620687     True  24.53133  61.18833   
        2009-05-28 08:00:00  1082620688     True  24.58200  61.23283   
        2009-05-28 14:00:00  1082620689     True  24.58250  61.23267   

                            sensor-type individual-taxon-canonical-name  \
traj_id DateTime                                                          
91732   2009-05-27 14:00:00         gps                    Larus fuscus   
        2009-05-27 20:00:00         gps                    Larus fuscus   
        2009-05-28 05:00:00         gps                    Larus fuscus   
        2009-05-28 08:00:00         gps                    Larus fuscus   
        2009-05-28 14:00:00         gps                    Larus fuscus   

                            individual-local-identifier  \
traj_id DateTime                                          
91732   2009-05-27 14:00:00                      91732A   
        2009-05-27 20:00:00                      91732A   
        2009-05-28 05:00:00                      91732A   
        2009-05-28 08:00:00                      91732A   
        2009-05-28 14:00:00                      91732A   

                                                                    study-name  \
traj_id DateTime                                                                 
91732   2009-05-27 14:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-27 20:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 05:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 08:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 14:00:00  Navigation experiments in lesser black-backed ...   

                             Distance_prev_to_curr  Distance_start_to_curr  \
traj_id DateTime                                                             
91732   2009-05-27 14:00:00                    NaN                     NaN   
        2009-05-27 20:00:00            1699.244398             1699.244398   
        2009-05-28 05:00:00            5632.120064             7238.295521   
        2009-05-28 08:00:00            5643.314949             1682.777895   
        2009-05-28 14:00:00              32.131494             1697.111327   

                             Speed_prev_to_curr  Acceleration_prev_to_curr  
traj_id DateTime                                                            
91732   2009-05-27 14:00:00                 NaN                        NaN  
        2009-05-27 20:00:00            0.078669                        NaN  
        2009-05-28 05:00:00            0.173831                   0.000003  
        2009-05-28 08:00:00            0.522529                   0.000032  
        2009-05-28 14:00:00            0.001488                  -0.000024

In [22]:
%%time

# Now lets check the acceleration of the atlantic dataset.
atlantic = spatial.create_acceleration_from_prev_column(atlantic)
atlantic.head()

CPU times: user 19.2 ms, sys: 455 µs, total: 19.7 ms
Wall time: 18.3 ms


Name      Date  Time Event  \
traj_id  DateTime                                                         
AL021873 1873-08-13 00:00:00              UNNAMED  18730813     0         
         1873-08-13 06:00:00              UNNAMED  18730813   600         
         1873-08-13 12:00:00              UNNAMED  18730813  1200         
         1873-08-13 18:00:00              UNNAMED  18730813  1800         
         1873-08-14 00:00:00              UNNAMED  18730814     0         

                             Status   lat   lon  Maximum Wind  \
traj_id  DateTime                                               
AL021873 1873-08-13 00:00:00     TS  13.9 -25.0            40   
         1873-08-13 06:00:00     TS  14.0 -26.0            40   
         1873-08-13 12:00:00     TS  14.0 -27.0            40   
         1873-08-13 18:00:00     TS  14.1 -28.1            40   
         1873-08-14 00:00:00     TS  14.1 -29.3            50   

                              Minimum Pressure  Low Wind NE  ...  \
traj_id  DateTime                                            ...   
AL021873 1873-08-13 00:00:00              -999         -999  ...   
         1873-08-13 06:00:00              -999         -999  ...   
         1873-08-13 12:00:00              -999         -999  ...   
         1873-08-13 18:00:00              -999         -999  ...   
         1873-08-14 00:00:00              -999         -999  ...   

                              Moderate Wind SW  Moderate Wind NW  \
traj_id  DateTime                                                  
AL021873 1873-08-13 00:00:00              -999              -999   
         1873-08-13 06:00:00              -999              -999   
         1873-08-13 12:00:00              -999              -999   
         1873-08-13 18:00:00              -999              -999   
         1873-08-14 00:00:00              -999              -999   

                              High Wind NE  High Wind SE  High Wind SW  \
traj_id  DateTime                                                        
AL021873 1873-08-13 00:00:00          -999          -999          -999   
         1873-08-13 06:00:00          -999          -999          -999   
         1873-08-13 12:00:00          -999          -999          -999   
         1873-08-13 18:00:00          -999          -999          -999   
         1873-08-14 00:00:00          -999          -999          -999   

                              High Wind NW  Distance_prev_to_curr  \
traj_id  DateTime                                                   
AL021873 1873-08-13 00:00:00          -999                    NaN   
         1873-08-13 06:00:00          -999          108486.658872   
         1873-08-13 12:00:00          -999          107891.881958   
         1873-08-13 18:00:00          -999          119175.045002   
         1873-08-14 00:00:00          -999          129413.676731   

                              Distance_start_to_curr  Speed_prev_to_curr  \
traj_id  DateTime                                                          
AL021873 1873-08-13 00:00:00                     NaN                 NaN   
         1873-08-13 06:00:00           108486.658872            5.022531   
         1873-08-13 12:00:00           216116.369259            4.994995   
         1873-08-13 18:00:00           335201.031514            5.517363   
         1873-08-14 00:00:00           464461.502447            5.991374   

                              Acceleration_prev_to_curr  
traj_id  DateTime                                        
AL021873 1873-08-13 00:00:00                        NaN  
         1873-08-13 06:00:00                        NaN  
         1873-08-13 12:00:00                  -0.000001  
         1873-08-13 18:00:00                   0.000024  
         1873-08-14 00:00:00                   0.000022  

[5 rows x 25 columns]

In [23]:
%%time

# Now lets check the jerk of the object between 2 consecutive
# points.
# First, we will do it for the geolife dataset.
geolife = spatial.create_jerk_from_prev_column(geolife)
geolife.head()

CPU times: user 37.8 ms, sys: 0 ns, total: 37.8 ms
Wall time: 36.1 ms


lat         lon  Distance_prev_to_curr  \
traj_id DateTime                                                            
5       2008-10-24 04:12:30  40.004155  116.321337                    NaN   
        2008-10-24 04:12:35  40.003834  116.321462              37.247654   
        2008-10-24 04:12:40  40.003783  116.321431               6.255518   
        2008-10-24 04:12:45  40.003690  116.321429              10.342531   
        2008-10-24 04:12:50  40.003589  116.321427              11.231979   

                             Distance_start_to_curr  Distance_to_(0, 0)  \
traj_id DateTime                                                          
5       2008-10-24 04:12:30                     NaN        1.221536e+07   
        2008-10-24 04:12:35               37.247654        1.221538e+07   
        2008-10-24 04:12:40               42.132251        1.221538e+07   
        2008-10-24 04:12:45               52.296064        1.221538e+07   
        2008-10-24 04:12:50               63.401465        1.221539e+07   

                             Within_15000000_m_from_(0, 0)  \
traj_id DateTime                                             
5       2008-10-24 04:12:30                           True   
        2008-10-24 04:12:35                           True   
        2008-10-24 04:12:40                           True   
        2008-10-24 04:12:45                           True   
        2008-10-24 04:12:50                           True   

                             Speed_prev_to_curr  Acceleration_prev_to_curr  \
traj_id DateTime                                                             
5       2008-10-24 04:12:30                 NaN                        NaN   
        2008-10-24 04:12:35            7.449531                        NaN   
        2008-10-24 04:12:40            1.251104                  -1.239685   
        2008-10-24 04:12:45            2.068506                   0.163481   
        2008-10-24 04:12:50            2.246396                   0.035578   

                             jerk_prev_to_curr  
traj_id DateTime                                
5       2008-10-24 04:12:30                NaN  
        2008-10-24 04:12:35                NaN  
        2008-10-24 04:12:40                NaN  
        2008-10-24 04:12:45           0.280633  
        2008-10-24 04:12:50          -0.025581

In [24]:
%%time

# Now lets check the jerk for the atlantic dataset.
atlantic = spatial.create_jerk_from_prev_column(atlantic)
atlantic.head()

CPU times: user 13.7 ms, sys: 7.12 ms, total: 20.8 ms
Wall time: 18.8 ms


Name      Date  Time Event  \
traj_id  DateTime                                                         
AL021873 1873-08-13 00:00:00              UNNAMED  18730813     0         
         1873-08-13 06:00:00              UNNAMED  18730813   600         
         1873-08-13 12:00:00              UNNAMED  18730813  1200         
         1873-08-13 18:00:00              UNNAMED  18730813  1800         
         1873-08-14 00:00:00              UNNAMED  18730814     0         

                             Status   lat   lon  Maximum Wind  \
traj_id  DateTime                                               
AL021873 1873-08-13 00:00:00     TS  13.9 -25.0            40   
         1873-08-13 06:00:00     TS  14.0 -26.0            40   
         1873-08-13 12:00:00     TS  14.0 -27.0            40   
         1873-08-13 18:00:00     TS  14.1 -28.1            40   
         1873-08-14 00:00:00     TS  14.1 -29.3            50   

                              Minimum Pressure  Low Wind NE  ...  \
traj_id  DateTime                                            ...   
AL021873 1873-08-13 00:00:00              -999         -999  ...   
         1873-08-13 06:00:00              -999         -999  ...   
         1873-08-13 12:00:00              -999         -999  ...   
         1873-08-13 18:00:00              -999         -999  ...   
         1873-08-14 00:00:00              -999         -999  ...   

                              Moderate Wind NW  High Wind NE  High Wind SE  \
traj_id  DateTime                                                            
AL021873 1873-08-13 00:00:00              -999          -999          -999   
         1873-08-13 06:00:00              -999          -999          -999   
         1873-08-13 12:00:00              -999          -999          -999   
         1873-08-13 18:00:00              -999          -999          -999   
         1873-08-14 00:00:00              -999          -999          -999   

                              High Wind SW  High Wind NW  \
traj_id  DateTime                                          
AL021873 1873-08-13 00:00:00          -999          -999   
         1873-08-13 06:00:00          -999          -999   
         1873-08-13 12:00:00          -999          -999   
         1873-08-13 18:00:00          -999          -999   
         1873-08-14 00:00:00          -999          -999   

                              Distance_prev_to_curr  Distance_start_to_curr  \
traj_id  DateTime                                                             
AL021873 1873-08-13 00:00:00                    NaN                     NaN   
         1873-08-13 06:00:00          108486.658872           108486.658872   
         1873-08-13 12:00:00          107891.881958           216116.369259   
         1873-08-13 18:00:00          119175.045002           335201.031514   
         1873-08-14 00:00:00          129413.676731           464461.502447   

                              Speed_prev_to_curr  Acceleration_prev_to_curr  \
traj_id  DateTime                                                             
AL021873 1873-08-13 00:00:00                 NaN                        NaN   
         1873-08-13 06:00:00            5.022531                        NaN   
         1873-08-13 12:00:00            4.994995                  -0.000001   
         1873-08-13 18:00:00            5.517363                   0.000024   
         1873-08-14 00:00:00            5.991374                   0.000022   

                              jerk_prev_to_curr  
traj_id  DateTime                                
AL021873 1873-08-13 00:00:00                NaN  
         1873-08-13 06:00:00                NaN  
         1873-08-13 12:00:00                NaN  
         1873-08-13 18:00:00       1.178636e-09  
         1873-08-14 00:00:00      -1.036478e-10  

[5 rows x 26 columns]

In [25]:
%%time

# Now, lets check the jerk for the seagulls dataset.
gulls = spatial.create_jerk_from_prev_column(gulls)
gulls.head()

CPU times: user 25.1 ms, sys: 145 µs, total: 25.3 ms
Wall time: 23.7 ms


event-id  visible       lon       lat  \
traj_id DateTime                                                       
91732   2009-05-27 14:00:00  1082620685     True  24.58617  61.24783   
        2009-05-27 20:00:00  1082620686     True  24.58217  61.23267   
        2009-05-28 05:00:00  1082620687     True  24.53133  61.18833   
        2009-05-28 08:00:00  1082620688     True  24.58200  61.23283   
        2009-05-28 14:00:00  1082620689     True  24.58250  61.23267   

                            sensor-type individual-taxon-canonical-name  \
traj_id DateTime                                                          
91732   2009-05-27 14:00:00         gps                    Larus fuscus   
        2009-05-27 20:00:00         gps                    Larus fuscus   
        2009-05-28 05:00:00         gps                    Larus fuscus   
        2009-05-28 08:00:00         gps                    Larus fuscus   
        2009-05-28 14:00:00         gps                    Larus fuscus   

                            individual-local-identifier  \
traj_id DateTime                                          
91732   2009-05-27 14:00:00                      91732A   
        2009-05-27 20:00:00                      91732A   
        2009-05-28 05:00:00                      91732A   
        2009-05-28 08:00:00                      91732A   
        2009-05-28 14:00:00                      91732A   

                                                                    study-name  \
traj_id DateTime                                                                 
91732   2009-05-27 14:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-27 20:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 05:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 08:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 14:00:00  Navigation experiments in lesser black-backed ...   

                             Distance_prev_to_curr  Distance_start_to_curr  \
traj_id DateTime                                                             
91732   2009-05-27 14:00:00                    NaN                     NaN   
        2009-05-27 20:00:00            1699.244398             1699.244398   
        2009-05-28 05:00:00            5632.120064             7238.295521   
        2009-05-28 08:00:00            5643.314949             1682.777895   
        2009-05-28 14:00:00              32.131494             1697.111327   

                             Speed_prev_to_curr  Acceleration_prev_to_curr  \
traj_id DateTime                                                             
91732   2009-05-27 14:00:00                 NaN                        NaN   
        2009-05-27 20:00:00            0.078669                        NaN   
        2009-05-28 05:00:00            0.173831                   0.000003   
        2009-05-28 08:00:00            0.522529                   0.000032   
        2009-05-28 14:00:00            0.001488                  -0.000024   

                             jerk_prev_to_curr  
traj_id DateTime                                
91732   2009-05-27 14:00:00                NaN  
        2009-05-27 20:00:00                NaN  
        2009-05-28 05:00:00                NaN  
        2009-05-28 08:00:00       2.717572e-09  
        2009-05-28 14:00:00      -2.611536e-09

In [26]:
%%time

# Now lets check the bearing between 2 consecutive
# points.
# First, we will do it for the geolife dataset.
geolife = spatial.create_bearing_column(geolife)
geolife.head()

CPU times: user 404 ms, sys: 27.9 ms, total: 432 ms
Wall time: 430 ms


lat         lon  Distance_prev_to_curr  \
traj_id DateTime                                                            
5       2008-10-24 04:12:30  40.004155  116.321337                    NaN   
        2008-10-24 04:12:35  40.003834  116.321462              37.247654   
        2008-10-24 04:12:40  40.003783  116.321431               6.255518   
        2008-10-24 04:12:45  40.003690  116.321429              10.342531   
        2008-10-24 04:12:50  40.003589  116.321427              11.231979   

                             Distance_start_to_curr  Distance_to_(0, 0)  \
traj_id DateTime                                                          
5       2008-10-24 04:12:30                     NaN        1.221536e+07   
        2008-10-24 04:12:35               37.247654        1.221538e+07   
        2008-10-24 04:12:40               42.132251        1.221538e+07   
        2008-10-24 04:12:45               52.296064        1.221538e+07   
        2008-10-24 04:12:50               63.401465        1.221539e+07   

                             Within_15000000_m_from_(0, 0)  \
traj_id DateTime                                             
5       2008-10-24 04:12:30                           True   
        2008-10-24 04:12:35                           True   
        2008-10-24 04:12:40                           True   
        2008-10-24 04:12:45                           True   
        2008-10-24 04:12:50                           True   

                             Speed_prev_to_curr  Acceleration_prev_to_curr  \
traj_id DateTime                                                             
5       2008-10-24 04:12:30                 NaN                        NaN   
        2008-10-24 04:12:35            7.449531                        NaN   
        2008-10-24 04:12:40            1.251104                  -1.239685   
        2008-10-24 04:12:45            2.068506                   0.163481   
        2008-10-24 04:12:50            2.246396                   0.035578   

                             jerk_prev_to_curr  Bearing_between_consecutive  
traj_id DateTime                                                             
5       2008-10-24 04:12:30                NaN                          NaN  
        2008-10-24 04:12:35                NaN                   163.390830  
        2008-10-24 04:12:40                NaN                  -155.032885  
        2008-10-24 04:12:45           0.280633                  -179.056241  
        2008-10-24 04:12:50          -0.025581                  -179.130981

In [27]:
%%time

# Now lets check the bearing for the atlantic dataset.
atlantic = spatial.create_bearing_column(atlantic)
atlantic.head()

CPU times: user 370 ms, sys: 235 ms, total: 605 ms
Wall time: 1.35 s


Name      Date  Time Event  \
traj_id  DateTime                                                         
AL021873 1873-08-13 00:00:00              UNNAMED  18730813     0         
         1873-08-13 06:00:00              UNNAMED  18730813   600         
         1873-08-13 12:00:00              UNNAMED  18730813  1200         
         1873-08-13 18:00:00              UNNAMED  18730813  1800         
         1873-08-14 00:00:00              UNNAMED  18730814     0         

                             Status   lat   lon  Maximum Wind  \
traj_id  DateTime                                               
AL021873 1873-08-13 00:00:00     TS  13.9 -25.0            40   
         1873-08-13 06:00:00     TS  14.0 -26.0            40   
         1873-08-13 12:00:00     TS  14.0 -27.0            40   
         1873-08-13 18:00:00     TS  14.1 -28.1            40   
         1873-08-14 00:00:00     TS  14.1 -29.3            50   

                              Minimum Pressure  Low Wind NE  ...  \
traj_id  DateTime                                            ...   
AL021873 1873-08-13 00:00:00              -999         -999  ...   
         1873-08-13 06:00:00              -999         -999  ...   
         1873-08-13 12:00:00              -999         -999  ...   
         1873-08-13 18:00:00              -999         -999  ...   
         1873-08-14 00:00:00              -999         -999  ...   

                              High Wind NE  High Wind SE  High Wind SW  \
traj_id  DateTime                                                        
AL021873 1873-08-13 00:00:00          -999          -999          -999   
         1873-08-13 06:00:00          -999          -999          -999   
         1873-08-13 12:00:00          -999          -999          -999   
         1873-08-13 18:00:00          -999          -999          -999   
         1873-08-14 00:00:00          -999          -999          -999   

                              High Wind NW  Distance_prev_to_curr  \
traj_id  DateTime                                                   
AL021873 1873-08-13 00:00:00          -999                    NaN   
         1873-08-13 06:00:00          -999          108486.658872   
         1873-08-13 12:00:00          -999          107891.881958   
         1873-08-13 18:00:00          -999          119175.045002   
         1873-08-14 00:00:00          -999          129413.676731   

                              Distance_start_to_curr  Speed_prev_to_curr  \
traj_id  DateTime                                                          
AL021873 1873-08-13 00:00:00                     NaN                 NaN   
         1873-08-13 06:00:00           108486.658872            5.022531   
         1873-08-13 12:00:00           216116.369259            4.994995   
         1873-08-13 18:00:00           335201.031514            5.517363   
         1873-08-14 00:00:00           464461.502447            5.991374   

                              Acceleration_prev_to_curr  jerk_prev_to_curr  \
traj_id  DateTime                                                            
AL021873 1873-08-13 00:00:00                        NaN                NaN   
         1873-08-13 06:00:00                        NaN                NaN   
         1873-08-13 12:00:00                  -0.000001                NaN   
         1873-08-13 18:00:00                   0.000024       1.178636e-09   
         1873-08-14 00:00:00                   0.000022      -1.036478e-10   

                              Bearing_between_consecutive  
traj_id  DateTime                                          
AL021873 1873-08-13 00:00:00                          NaN  
         1873-08-13 06:00:00                   -83.996671  
         1873-08-13 12:00:00                   -89.879036  
         1873-08-13 18:00:00                   -84.512938  
         1873-08-14 00:00:00                   -89.853826  

[5 rows x 27 columns]

In [28]:
%%time

# Now lets check the bearing for the seagulls dataset.
gulls = spatial.create_bearing_column(gulls)
gulls.head()


CPU times: user 178 ms, sys: 71.9 ms, total: 250 ms
Wall time: 487 ms


event-id  visible       lon       lat  \
traj_id DateTime                                                       
91732   2009-05-27 14:00:00  1082620685     True  24.58617  61.24783   
        2009-05-27 20:00:00  1082620686     True  24.58217  61.23267   
        2009-05-28 05:00:00  1082620687     True  24.53133  61.18833   
        2009-05-28 08:00:00  1082620688     True  24.58200  61.23283   
        2009-05-28 14:00:00  1082620689     True  24.58250  61.23267   

                            sensor-type individual-taxon-canonical-name  \
traj_id DateTime                                                          
91732   2009-05-27 14:00:00         gps                    Larus fuscus   
        2009-05-27 20:00:00         gps                    Larus fuscus   
        2009-05-28 05:00:00         gps                    Larus fuscus   
        2009-05-28 08:00:00         gps                    Larus fuscus   
        2009-05-28 14:00:00         gps                    Larus fuscus   

                            individual-local-identifier  \
traj_id DateTime                                          
91732   2009-05-27 14:00:00                      91732A   
        2009-05-27 20:00:00                      91732A   
        2009-05-28 05:00:00                      91732A   
        2009-05-28 08:00:00                      91732A   
        2009-05-28 14:00:00                      91732A   

                                                                    study-name  \
traj_id DateTime                                                                 
91732   2009-05-27 14:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-27 20:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 05:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 08:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 14:00:00  Navigation experiments in lesser black-backed ...   

                             Distance_prev_to_curr  Distance_start_to_curr  \
traj_id DateTime                                                             
91732   2009-05-27 14:00:00                    NaN                     NaN   
        2009-05-27 20:00:00            1699.244398             1699.244398   
        2009-05-28 05:00:00            5632.120064             7238.295521   
        2009-05-28 08:00:00            5643.314949             1682.777895   
        2009-05-28 14:00:00              32.131494             1697.111327   

                             Speed_prev_to_curr  Acceleration_prev_to_curr  \
traj_id DateTime                                                             
91732   2009-05-27 14:00:00                 NaN                        NaN   
        2009-05-27 20:00:00            0.078669                        NaN   
        2009-05-28 05:00:00            0.173831                   0.000003   
        2009-05-28 08:00:00            0.522529                   0.000032   
        2009-05-28 14:00:00            0.001488                  -0.000024   

                             jerk_prev_to_curr  Bearing_between_consecutive  
traj_id DateTime                                                             
91732   2009-05-27 14:00:00                NaN                          NaN  
        2009-05-27 20:00:00                NaN                  -172.763287  
        2009-05-28 05:00:00                NaN                  -151.070593  
        2009-05-28 08:00:00       2.717572e-09                    28.716637  
        2009-05-28 14:00:00      -2.611536e-09                   123.620959

In [29]:
%%time

# Now, lets calculate the bearing rate between 2 consecutive points.
# First, we will do it for the geolife dataset.
geolife = spatial.create_bearing_rate_column(geolife)
geolife.head()

CPU times: user 26.9 ms, sys: 6.6 ms, total: 33.5 ms
Wall time: 32.4 ms


lat         lon  Distance_prev_to_curr  \
traj_id DateTime                                                            
5       2008-10-24 04:12:30  40.004155  116.321337                    NaN   
        2008-10-24 04:12:35  40.003834  116.321462              37.247654   
        2008-10-24 04:12:40  40.003783  116.321431               6.255518   
        2008-10-24 04:12:45  40.003690  116.321429              10.342531   
        2008-10-24 04:12:50  40.003589  116.321427              11.231979   

                             Distance_start_to_curr  Distance_to_(0, 0)  \
traj_id DateTime                                                          
5       2008-10-24 04:12:30                     NaN        1.221536e+07   
        2008-10-24 04:12:35               37.247654        1.221538e+07   
        2008-10-24 04:12:40               42.132251        1.221538e+07   
        2008-10-24 04:12:45               52.296064        1.221538e+07   
        2008-10-24 04:12:50               63.401465        1.221539e+07   

                             Within_15000000_m_from_(0, 0)  \
traj_id DateTime                                             
5       2008-10-24 04:12:30                           True   
        2008-10-24 04:12:35                           True   
        2008-10-24 04:12:40                           True   
        2008-10-24 04:12:45                           True   
        2008-10-24 04:12:50                           True   

                             Speed_prev_to_curr  Acceleration_prev_to_curr  \
traj_id DateTime                                                             
5       2008-10-24 04:12:30                 NaN                        NaN   
        2008-10-24 04:12:35            7.449531                        NaN   
        2008-10-24 04:12:40            1.251104                  -1.239685   
        2008-10-24 04:12:45            2.068506                   0.163481   
        2008-10-24 04:12:50            2.246396                   0.035578   

                             jerk_prev_to_curr  Bearing_between_consecutive  \
traj_id DateTime                                                              
5       2008-10-24 04:12:30                NaN                          NaN   
        2008-10-24 04:12:35                NaN                   163.390830   
        2008-10-24 04:12:40                NaN                  -155.032885   
        2008-10-24 04:12:45           0.280633                  -179.056241   
        2008-10-24 04:12:50          -0.025581                  -179.130981   

                             Bearing_rate_from_prev  
traj_id DateTime                                     
5       2008-10-24 04:12:30                     NaN  
        2008-10-24 04:12:35                     NaN  
        2008-10-24 04:12:40              -63.684743  
        2008-10-24 04:12:45               -4.804671  
        2008-10-24 04:12:50               -0.014948

In [30]:
%%time

# Now lets calculate the bearing rate for the atlantic dataset.
atlantic = spatial.create_bearing_rate_column(atlantic)
atlantic.head()

CPU times: user 18.9 ms, sys: 437 µs, total: 19.3 ms
Wall time: 18.4 ms


Name      Date  Time Event  \
traj_id  DateTime                                                         
AL021873 1873-08-13 00:00:00              UNNAMED  18730813     0         
         1873-08-13 06:00:00              UNNAMED  18730813   600         
         1873-08-13 12:00:00              UNNAMED  18730813  1200         
         1873-08-13 18:00:00              UNNAMED  18730813  1800         
         1873-08-14 00:00:00              UNNAMED  18730814     0         

                             Status   lat   lon  Maximum Wind  \
traj_id  DateTime                                               
AL021873 1873-08-13 00:00:00     TS  13.9 -25.0            40   
         1873-08-13 06:00:00     TS  14.0 -26.0            40   
         1873-08-13 12:00:00     TS  14.0 -27.0            40   
         1873-08-13 18:00:00     TS  14.1 -28.1            40   
         1873-08-14 00:00:00     TS  14.1 -29.3            50   

                              Minimum Pressure  Low Wind NE  ...  \
traj_id  DateTime                                            ...   
AL021873 1873-08-13 00:00:00              -999         -999  ...   
         1873-08-13 06:00:00              -999         -999  ...   
         1873-08-13 12:00:00              -999         -999  ...   
         1873-08-13 18:00:00              -999         -999  ...   
         1873-08-14 00:00:00              -999         -999  ...   

                              High Wind SE  High Wind SW  High Wind NW  \
traj_id  DateTime                                                        
AL021873 1873-08-13 00:00:00          -999          -999          -999   
         1873-08-13 06:00:00          -999          -999          -999   
         1873-08-13 12:00:00          -999          -999          -999   
         1873-08-13 18:00:00          -999          -999          -999   
         1873-08-14 00:00:00          -999          -999          -999   

                              Distance_prev_to_curr  Distance_start_to_curr  \
traj_id  DateTime                                                             
AL021873 1873-08-13 00:00:00                    NaN                     NaN   
         1873-08-13 06:00:00          108486.658872           108486.658872   
         1873-08-13 12:00:00          107891.881958           216116.369259   
         1873-08-13 18:00:00          119175.045002           335201.031514   
         1873-08-14 00:00:00          129413.676731           464461.502447   

                              Speed_prev_to_curr  Acceleration_prev_to_curr  \
traj_id  DateTime                                                             
AL021873 1873-08-13 00:00:00                 NaN                        NaN   
         1873-08-13 06:00:00            5.022531                        NaN   
         1873-08-13 12:00:00            4.994995                  -0.000001   
         1873-08-13 18:00:00            5.517363                   0.000024   
         1873-08-14 00:00:00            5.991374                   0.000022   

                              jerk_prev_to_curr  Bearing_between_consecutive  \
traj_id  DateTime                                                              
AL021873 1873-08-13 00:00:00                NaN                          NaN   
         1873-08-13 06:00:00                NaN                   -83.996671   
         1873-08-13 12:00:00                NaN                   -89.879036   
         1873-08-13 18:00:00       1.178636e-09                   -84.512938   
         1873-08-14 00:00:00      -1.036478e-10                   -89.853826   

                              Bearing_rate_from_prev  
traj_id  DateTime                                     
AL021873 1873-08-13 00:00:00                     NaN  
         1873-08-13 06:00:00                     NaN  
         1873-08-13 12:00:00               -0.000272  
         1873-08-13 18:00:00                0.000248  
         1873-08-14 00:00:00               -0.000247  

[5 rows x 28

In [31]:
%%time

# Now lets calculate the bearing rate for the seagulls dataset.
gulls = spatial.create_bearing_rate_column(gulls)
gulls.head()

CPU times: user 18.2 ms, sys: 30 µs, total: 18.3 ms
Wall time: 16.6 ms


event-id  visible       lon       lat  \
traj_id DateTime                                                       
91732   2009-05-27 14:00:00  1082620685     True  24.58617  61.24783   
        2009-05-27 20:00:00  1082620686     True  24.58217  61.23267   
        2009-05-28 05:00:00  1082620687     True  24.53133  61.18833   
        2009-05-28 08:00:00  1082620688     True  24.58200  61.23283   
        2009-05-28 14:00:00  1082620689     True  24.58250  61.23267   

                            sensor-type individual-taxon-canonical-name  \
traj_id DateTime                                                          
91732   2009-05-27 14:00:00         gps                    Larus fuscus   
        2009-05-27 20:00:00         gps                    Larus fuscus   
        2009-05-28 05:00:00         gps                    Larus fuscus   
        2009-05-28 08:00:00         gps                    Larus fuscus   
        2009-05-28 14:00:00         gps                    Larus fuscus   

                            individual-local-identifier  \
traj_id DateTime                                          
91732   2009-05-27 14:00:00                      91732A   
        2009-05-27 20:00:00                      91732A   
        2009-05-28 05:00:00                      91732A   
        2009-05-28 08:00:00                      91732A   
        2009-05-28 14:00:00                      91732A   

                                                                    study-name  \
traj_id DateTime                                                                 
91732   2009-05-27 14:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-27 20:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 05:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 08:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 14:00:00  Navigation experiments in lesser black-backed ...   

                             Distance_prev_to_curr  Distance_start_to_curr  \
traj_id DateTime                                                             
91732   2009-05-27 14:00:00                    NaN                     NaN   
        2009-05-27 20:00:00            1699.244398             1699.244398   
        2009-05-28 05:00:00            5632.120064             7238.295521   
        2009-05-28 08:00:00            5643.314949             1682.777895   
        2009-05-28 14:00:00              32.131494             1697.111327   

                             Speed_prev_to_curr  Acceleration_prev_to_curr  \
traj_id DateTime                                                             
91732   2009-05-27 14:00:00                 NaN                        NaN   
        2009-05-27 20:00:00            0.078669                        NaN   
        2009-05-28 05:00:00            0.173831                   0.000003   
        2009-05-28 08:00:00            0.522529                   0.000032   
        2009-05-28 14:00:00            0.001488                  -0.000024   

                             jerk_prev_to_curr  Bearing_between_consecutive  \
traj_id DateTime                                                              
91732   2009-05-27 14:00:00                NaN                          NaN   
        2009-05-27 20:00:00                NaN                  -172.763287   
        2009-05-28 05:00:00                NaN                  -151.070593   
        2009-05-28 08:00:00       2.717572e-09                    28.716637   
        2009-05-28 14:00:00      -2.611536e-09                   123.620959   

                             Bearing_rate_from_prev  
traj_id DateTime                                     
91732   2009-05-27 14:00:00                     NaN  
        2009-05-27 20:00:00                     NaN  
        2009-05-28 05:00:00                0.000670  
        2009-05-28 08:00:00                0.016647  
        2009-05-28 14:00:00            

In [32]:
%%time

# Now, lets calculate the rate of bearing rate between 2
# consecutive points.
geolife = spatial.create_rate_of_bearing_rate_column(geolife)
geolife.head()


CPU times: user 22.4 ms, sys: 153 µs, total: 22.5 ms
Wall time: 20.9 ms


lat         lon  Distance_prev_to_curr  \
traj_id DateTime                                                            
5       2008-10-24 04:12:30  40.004155  116.321337                    NaN   
        2008-10-24 04:12:35  40.003834  116.321462              37.247654   
        2008-10-24 04:12:40  40.003783  116.321431               6.255518   
        2008-10-24 04:12:45  40.003690  116.321429              10.342531   
        2008-10-24 04:12:50  40.003589  116.321427              11.231979   

                             Distance_start_to_curr  Distance_to_(0, 0)  \
traj_id DateTime                                                          
5       2008-10-24 04:12:30                     NaN        1.221536e+07   
        2008-10-24 04:12:35               37.247654        1.221538e+07   
        2008-10-24 04:12:40               42.132251        1.221538e+07   
        2008-10-24 04:12:45               52.296064        1.221538e+07   
        2008-10-24 04:12:50               63.401465        1.221539e+07   

                             Within_15000000_m_from_(0, 0)  \
traj_id DateTime                                             
5       2008-10-24 04:12:30                           True   
        2008-10-24 04:12:35                           True   
        2008-10-24 04:12:40                           True   
        2008-10-24 04:12:45                           True   
        2008-10-24 04:12:50                           True   

                             Speed_prev_to_curr  Acceleration_prev_to_curr  \
traj_id DateTime                                                             
5       2008-10-24 04:12:30                 NaN                        NaN   
        2008-10-24 04:12:35            7.449531                        NaN   
        2008-10-24 04:12:40            1.251104                  -1.239685   
        2008-10-24 04:12:45            2.068506                   0.163481   
        2008-10-24 04:12:50            2.246396                   0.035578   

                             jerk_prev_to_curr  Bearing_between_consecutive  \
traj_id DateTime                                                              
5       2008-10-24 04:12:30                NaN                          NaN   
        2008-10-24 04:12:35                NaN                   163.390830   
        2008-10-24 04:12:40                NaN                  -155.032885   
        2008-10-24 04:12:45           0.280633                  -179.056241   
        2008-10-24 04:12:50          -0.025581                  -179.130981   

                             Bearing_rate_from_prev  \
traj_id DateTime                                      
5       2008-10-24 04:12:30                     NaN   
        2008-10-24 04:12:35                     NaN   
        2008-10-24 04:12:40              -63.684743   
        2008-10-24 04:12:45               -4.804671   
        2008-10-24 04:12:50               -0.014948   

                             Rate_of_bearing_rate_from_prev  
traj_id DateTime                                             
5       2008-10-24 04:12:30                             NaN  
        2008-10-24 04:12:35                             NaN  
        2008-10-24 04:12:40                             NaN  
        2008-10-24 04:12:45                       11.776014  
        2008-10-24 04:12:50                        0.957945

In [33]:
%%time

# Now, lets calculate the rate of bearing rate for the atlantic
# dataset.
atlantic = spatial.create_rate_of_bearing_rate_column(atlantic)
atlantic.head()

CPU times: user 12.3 ms, sys: 39 µs, total: 12.4 ms
Wall time: 11.2 ms


Name      Date  Time Event  \
traj_id  DateTime                                                         
AL021873 1873-08-13 00:00:00              UNNAMED  18730813     0         
         1873-08-13 06:00:00              UNNAMED  18730813   600         
         1873-08-13 12:00:00              UNNAMED  18730813  1200         
         1873-08-13 18:00:00              UNNAMED  18730813  1800         
         1873-08-14 00:00:00              UNNAMED  18730814     0         

                             Status   lat   lon  Maximum Wind  \
traj_id  DateTime                                               
AL021873 1873-08-13 00:00:00     TS  13.9 -25.0            40   
         1873-08-13 06:00:00     TS  14.0 -26.0            40   
         1873-08-13 12:00:00     TS  14.0 -27.0            40   
         1873-08-13 18:00:00     TS  14.1 -28.1            40   
         1873-08-14 00:00:00     TS  14.1 -29.3            50   

                              Minimum Pressure  Low Wind NE  ...  \
traj_id  DateTime                                            ...   
AL021873 1873-08-13 00:00:00              -999         -999  ...   
         1873-08-13 06:00:00              -999         -999  ...   
         1873-08-13 12:00:00              -999         -999  ...   
         1873-08-13 18:00:00              -999         -999  ...   
         1873-08-14 00:00:00              -999         -999  ...   

                              High Wind SW  High Wind NW  \
traj_id  DateTime                                          
AL021873 1873-08-13 00:00:00          -999          -999   
         1873-08-13 06:00:00          -999          -999   
         1873-08-13 12:00:00          -999          -999   
         1873-08-13 18:00:00          -999          -999   
         1873-08-14 00:00:00          -999          -999   

                              Distance_prev_to_curr  Distance_start_to_curr  \
traj_id  DateTime                                                             
AL021873 1873-08-13 00:00:00                    NaN                     NaN   
         1873-08-13 06:00:00          108486.658872           108486.658872   
         1873-08-13 12:00:00          107891.881958           216116.369259   
         1873-08-13 18:00:00          119175.045002           335201.031514   
         1873-08-14 00:00:00          129413.676731           464461.502447   

                              Speed_prev_to_curr  Acceleration_prev_to_curr  \
traj_id  DateTime                                                             
AL021873 1873-08-13 00:00:00                 NaN                        NaN   
         1873-08-13 06:00:00            5.022531                        NaN   
         1873-08-13 12:00:00            4.994995                  -0.000001   
         1873-08-13 18:00:00            5.517363                   0.000024   
         1873-08-14 00:00:00            5.991374                   0.000022   

                              jerk_prev_to_curr  Bearing_between_consecutive  \
traj_id  DateTime                                                              
AL021873 1873-08-13 00:00:00                NaN                          NaN   
         1873-08-13 06:00:00                NaN                   -83.996671   
         1873-08-13 12:00:00                NaN                   -89.879036   
         1873-08-13 18:00:00       1.178636e-09                   -84.512938   
         1873-08-14 00:00:00      -1.036478e-10                   -89.853826   

                              Bearing_rate_from_prev  \
traj_id  DateTime                                      
AL021873 1873-08-13 00:00:00                     NaN   
         1873-08-13 06:00:00                     NaN   
         1873-08-13 12:00:00               -0.000272   
         1873-08-13 18:00:00                0.000248   
         1873-08-14 00:00:00               -0.000247   

                              Rate_of_bearing_rate_from_prev  
traj_id  DateTime                       

In [34]:
%%time

# Now lets calculate the rate of bearing rate for the seagulls
# dataset.
gulls = spatial.create_rate_of_bearing_rate_column(gulls)
gulls.head()

CPU times: user 17.2 ms, sys: 2.95 ms, total: 20.2 ms
Wall time: 18.6 ms


event-id  visible       lon       lat  \
traj_id DateTime                                                       
91732   2009-05-27 14:00:00  1082620685     True  24.58617  61.24783   
        2009-05-27 20:00:00  1082620686     True  24.58217  61.23267   
        2009-05-28 05:00:00  1082620687     True  24.53133  61.18833   
        2009-05-28 08:00:00  1082620688     True  24.58200  61.23283   
        2009-05-28 14:00:00  1082620689     True  24.58250  61.23267   

                            sensor-type individual-taxon-canonical-name  \
traj_id DateTime                                                          
91732   2009-05-27 14:00:00         gps                    Larus fuscus   
        2009-05-27 20:00:00         gps                    Larus fuscus   
        2009-05-28 05:00:00         gps                    Larus fuscus   
        2009-05-28 08:00:00         gps                    Larus fuscus   
        2009-05-28 14:00:00         gps                    Larus fuscus   

                            individual-local-identifier  \
traj_id DateTime                                          
91732   2009-05-27 14:00:00                      91732A   
        2009-05-27 20:00:00                      91732A   
        2009-05-28 05:00:00                      91732A   
        2009-05-28 08:00:00                      91732A   
        2009-05-28 14:00:00                      91732A   

                                                                    study-name  \
traj_id DateTime                                                                 
91732   2009-05-27 14:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-27 20:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 05:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 08:00:00  Navigation experiments in lesser black-backed ...   
        2009-05-28 14:00:00  Navigation experiments in lesser black-backed ...   

                             Distance_prev_to_curr  Distance_start_to_curr  \
traj_id DateTime                                                             
91732   2009-05-27 14:00:00                    NaN                     NaN   
        2009-05-27 20:00:00            1699.244398             1699.244398   
        2009-05-28 05:00:00            5632.120064             7238.295521   
        2009-05-28 08:00:00            5643.314949             1682.777895   
        2009-05-28 14:00:00              32.131494             1697.111327   

                             Speed_prev_to_curr  Acceleration_prev_to_curr  \
traj_id DateTime                                                             
91732   2009-05-27 14:00:00                 NaN                        NaN   
        2009-05-27 20:00:00            0.078669                        NaN   
        2009-05-28 05:00:00            0.173831                   0.000003   
        2009-05-28 08:00:00            0.522529                   0.000032   
        2009-05-28 14:00:00            0.001488                  -0.000024   

                             jerk_prev_to_curr  Bearing_between_consecutive  \
traj_id DateTime                                                              
91732   2009-05-27 14:00:00                NaN                          NaN   
        2009-05-27 20:00:00                NaN                  -172.763287   
        2009-05-28 05:00:00                NaN                  -151.070593   
        2009-05-28 08:00:00       2.717572e-09                    28.716637   
        2009-05-28 14:00:00      -2.611536e-09                   123.620959   

                             Bearing_rate_from_prev  \
traj_id DateTime                                      
91732   2009-05-27 14:00:00                     NaN   
        2009-05-27 20:00:00                     NaN   
        2009-05-28 05:00:00                0.000670   
        2009-05-28 08:00:00                0.016647   
        2009-05-28 14:00:00      

In [35]:
%%time

# Now, lets get the start locations of all the trajectories
# in the atlantic dataset.
spatial.get_start_location(atlantic)

CPU times: user 510 ms, sys: 553 ms, total: 1.06 s
Wall time: 3.95 s


,lat,lon
traj_id,,
AL041861,28.5,-50.0
AL041866,15.0,-29.5
AL071861,35.3,-75.3
AL061865,25.5,-69.5
AL091869,17.0,-67.5
...,...,...
AL041926,13.8,-41.2
AL142012,15.5,-38.0
AL201969,16.0,-47.0


In [36]:
%%time

# Now, lets get the end location of a specific trajectory
# from the atlantic dataset.
spatial.get_end_location(atlantic, 'AL011851')

CPU times: user 34.4 ms, sys: 12.2 ms, total: 46.6 ms
Wall time: 45.1 ms


(31.0, -100.2)

In [37]:
%%time

# Now, lets get the distance travelled by a specific trajectory
# in the atlantic dataset.
spatial.get_distance_travelled_by_traj_id(atlantic, 'AL011851')

CPU times: user 32.1 ms, sys: 4.57 ms, total: 36.7 ms
Wall time: 35.4 ms


716547.1062403042

In [38]:
%%time

# Now, lets get the number of unique locations visited by
# all the trajectories in the atlantic dataset.
spatial.get_number_of_locations(atlantic)

CPU times: user 487 ms, sys: 586 ms, total: 1.07 s
Wall time: 3.27 s


,Number of Unique Coordinates
traj_id,
AL041861,1
AL041866,1
AL071861,1
AL061865,1
AL091869,1
...,...
AL041926,95
AL142012,96
AL201969,99


In [39]:
end = time.time()
print(f"Total execution time is {end - start} seconds.")

Total execution time is 42.28119897842407 seconds.
